2. Простой пример на Python: клоны геометрических фигур

In [1]:
from __future__ import annotations
from abc import ABC, abstractmethod
import copy

# ---------- Prototype ----------
class Shape(ABC):
    def __init__(self, x: int, y: int, color: str) -> None:
        self.x, self.y, self.color = x, y, color

    @abstractmethod
    def clone(self) -> "Shape": ...

class Circle(Shape):
    def __init__(self, x: int, y: int, radius: int, color: str = "black") -> None:
        super().__init__(x, y, color)
        self.radius = radius

    def clone(self) -> "Circle":
        # copy.deepcopy → рекурсивная копия, подходит, если есть вложенные объекты
        return copy.deepcopy(self)

    def draw(self) -> None:
        print(f"⚪ Circle @({self.x},{self.y}) r={self.radius} color={self.color}")

class Rectangle(Shape):
    def __init__(self, x: int, y: int, w: int, h: int, color: str = "black") -> None:
        super().__init__(x, y, color)
        self.w, self.h = w, h

    def clone(self) -> "Rectangle":
        return copy.deepcopy(self)

    def draw(self) -> None:
        print(f"▭ Rect @({self.x},{self.y}) {self.w}×{self.h} color={self.color}")

# ---------- Клиентский код ----------
if __name__ == "__main__":
    circle_proto = Circle(0, 0, 10, "red")
    rect_proto   = Rectangle(0, 0, 20, 10, "blue")

    # Клонируем и слегка правим координаты
    shapes = [
        circle_proto.clone(),
        rect_proto.clone(),
        circle_proto.clone(),
    ]
    shapes[1].x, shapes[1].y = 5, 5
    shapes[2].x = 15

    for s in shapes:
        s.draw()


⚪ Circle @(0,0) r=10 color=red
▭ Rect @(5,5) 20×10 color=blue
⚪ Circle @(15,0) r=10 color=red


3. Сложный пример: RPG‑игра с реестром прототипов NPC
Задача: В игре сотни неигровых персонажей (NPC) с разными классами, снаряжением и AI‑поведением.
Создавать каждого через конструкторы дорого (чтение JSON‑ресурсов, загрузка 3‑D модели).
Решение — подготовить реестр прототипов при старте и быстро клонировать.

In [2]:
"""
RPG‑NPC Prototype Registry.
Показывает: (1) глубокое копирование сложного графа объектов,
(2) изменение клонов без влияния на прототип.
"""
from __future__ import annotations
from abc import ABC, abstractmethod
import copy
from dataclasses import dataclass, field
from typing import Dict, Type

# ---------- Составные части ----------
@dataclass
class Stats:
    hp: int
    mp: int
    strength: int
    agility: int

@dataclass
class Item:
    name: str
    damage: int = 0
    defense: int = 0

class AI(ABC):
    @abstractmethod
    def act(self) -> None: ...

class AggressiveAI(AI):
    def act(self) -> None:
        print("⚔️  Атакуем ближайшую цель!")

class DefensiveAI(AI):
    def act(self) -> None:
        print("🛡️  Обороняемся и ищем укрытие.")

# ---------- Prototype ----------
class NPC(ABC):
    def __init__(self, name: str, stats: Stats, equipment: list[Item], ai: AI) -> None:
        self.name = name
        self.stats = stats
        self.equipment = equipment
        self.ai = ai

    @abstractmethod
    def clone(self) -> "NPC": ...

    def show(self) -> None:
        eq = ", ".join(i.name for i in self.equipment)
        print(f"{self.__class__.__name__} «{self.name}» [{self.stats.hp} HP] → {eq}")

# ---------- Concrete Prototypes ----------
class Soldier(NPC):
    def clone(self) -> "Soldier":
        return copy.deepcopy(self)

class Archer(NPC):
    def clone(self) -> "Archer":
        return copy.deepcopy(self)

class Mage(NPC):
    def clone(self) -> "Mage":
        return copy.deepcopy(self)

# ---------- Реестр прототипов ----------
class PrototypeRegistry:
    def __init__(self) -> None:
        self._prototypes: Dict[str, NPC] = {}

    def register(self, key: str, prototype: NPC) -> None:
        self._prototypes[key] = prototype

    def clone(self, key: str, **attrs) -> NPC:
        proto = self._prototypes[key]
        npc = proto.clone()
        # Позволяем сразу кастомизировать клон через kwargs
        for k, v in attrs.items():
            setattr(npc, k, v)
        return npc

# ---------- Заполняем реестр при старте игры ----------
registry = PrototypeRegistry()

registry.register(
    "soldier",
    Soldier(
        "Base Soldier",
        Stats(100, 0, 15, 10),
        [Item("Sword", damage=10), Item("Shield", defense=5)],
        AggressiveAI(),
    ),
)
registry.register(
    "archer",
    Archer(
        "Base Archer",
        Stats(80, 0, 10, 15),
        [Item("Bow", damage=12)],
        DefensiveAI(),
    ),
)
registry.register(
    "mage",
    Mage(
        "Base Mage",
        Stats(60, 120, 5, 12),
        [Item("Staff", damage=8)],
        DefensiveAI(),
    ),
)

# ---------- Клиентский код ----------
if __name__ == "__main__":
    # Порождаем волну врагов
    wave = [
        registry.clone("soldier", name="Grunt‑A"),
        registry.clone("soldier", name="Grunt‑B"),
        registry.clone("archer", name="Archer‑A"),
        registry.clone("mage",   name="Mage‑Boss", stats=Stats(120, 200, 10, 14)),
    ]

    for npc in wave:
        npc.show()
        npc.ai.act()


Soldier «Grunt‑A» [100 HP] → Sword, Shield
⚔️  Атакуем ближайшую цель!
Soldier «Grunt‑B» [100 HP] → Sword, Shield
⚔️  Атакуем ближайшую цель!
Archer «Archer‑A» [80 HP] → Bow
🛡️  Обороняемся и ищем укрытие.
Mage «Mage‑Boss» [120 HP] → Staff
🛡️  Обороняемся и ищем укрытие.
